In [ ]:
keep 
atr_14  , OBV

drop

'ema_cross_up', 'macd_cross_up', 'oversold_reversal', 'overbought_reversal', 'trending_market'

In [ ]:
drop_lstm_cnn = [
    'open', 'high', 'low', 'typical_price', 'EMA_21', 'SMA_20',
    'vwap_24h', 'close_4h', 'bollinger_upper', 'bollinger_lower',
    'resistance_level', 'support_level', 'high_low', 'high_close', 'low_close',
    'true_range', 'volume_mean_20', 'MACD_line', 'MACD_signal',
    'volatility_regime', 'trending_market', 'above_sma50', 'ema7_above_ema21',
    'rsi_overbought', 'stoch_oversold', 'cci_oversold',
    'vol_spike_1_5x', 'near_upper_band', 'near_lower_band',
    'break_upper_band', 'break_lower_band', 'rsi_oversold',
    'above_sma20', 'macd_positive', 'volume_breakout', 'volume_breakdown',
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5', 'bullish_scenario_6',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3',
    'bearish_scenario_4', 'bearish_scenario_6','ema_cross_up', 'macd_cross_up', 'oversold_reversal', 'overbought_reversal'
]

In [18]:
# =============================================================
#  CNN–LSTM Optuna Trainer  (GPU, XLA, big batches)
# =============================================================
import os, json, gc, warnings, optuna
from datetime import datetime
from pathlib import Path
from typing import Dict, Tuple

import numpy as np
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (precision_score, recall_score,
                             f1_score, accuracy_score)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from optuna_integration.tfkeras import TFKerasPruningCallback

# ───── Runtime setup ─────
warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
tf.config.optimizer.set_jit(True)            # XLA on

SEED, VAL_FRAC, ALPHA = 42, 0.20, 2.0
N_TRIALS, TIMEOUT      = 100, 3 * 60 * 60    # 2h wall limit

CSV_PATH = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction"
                r"\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")
SCALER_PKL = "cnn_lstm_scaler.pkl"

DROP_COLS = [  # duplicates removed
    "open","high","low","typical_price","EMA_21","SMA_20","close_4h",
    "bollinger_upper","bollinger_lower","resistance_level","support_level",
    "high_low","high_close","low_close","true_range","volume_mean_20",
    "MACD_line","MACD_signal","volatility_regime","trending_market",
    "above_sma50","ema7_above_ema21","rsi_overbought","stoch_oversold",
    "cci_oversold","near_upper_band","near_lower_band","break_upper_band",
    "break_lower_band","rsi_oversold","above_sma20","macd_positive",
    "volume_breakout","volume_breakdown","bullish_scenario_1","bullish_scenario_2",
    "bullish_scenario_3","bullish_scenario_4","bullish_scenario_5","bullish_scenario_6",
    "bearish_scenario_1","bearish_scenario_2","bearish_scenario_3","bearish_scenario_4",
    "bearish_scenario_6","ema_cross_up","macd_cross_up",
    "oversold_reversal","overbought_reversal","close"
]

np.random.seed(SEED)
tf.random.set_seed(SEED)
for g in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(g, True)

# ───── Data ─────
df = (pd.read_csv(CSV_PATH, index_col=0, parse_dates=True)
        .loc["2018-01-01":]
        .drop(columns=[c for c in DROP_COLS if c in df.columns])
        .dropna(subset=["target"]).dropna())

X_raw = df.drop(columns="target").values.astype(np.float32)
y_raw = df["target"].astype(int).values
n_feat = X_raw.shape[1]

split = int(len(df) * (1 - VAL_FRAC))
scaler = StandardScaler().fit(X_raw[:split])
joblib.dump(scaler, SCALER_PKL)

X_tr_raw = scaler.transform(X_raw[:split]);  y_tr_raw = y_raw[:split]
X_va_raw = scaler.transform(X_raw[split:]);   y_va_raw = y_raw[split:]

pos   = y_tr_raw.mean()
W0, W1 = 1.0, np.float32((1 - pos) / pos) if pos else np.float32(1.0)

# ───── Window cache ─────
_CACHE: Dict[Tuple[int,int,int], Tuple[np.ndarray,np.ndarray]] = {}
def make_windows(arr, lab, win):
    k = (len(arr), win, arr.shape[1])
    if k in _CACHE: return _CACHE[k]
    xs, ys = [], []
    for i in range(win, len(arr)):
        xs.append(arr[i-win:i]); ys.append(lab[i])
    Xw, yw = np.asarray(xs,np.float32), np.asarray(ys,np.int8)
    if Xw.nbytes + yw.nbytes < 1e9:
        _CACHE[k] = (Xw, yw)
    return Xw, yw

def wf1(y, p, alpha=ALPHA):
    hat = (p >= .5).astype(int)
    pr  = precision_score(y, hat, zero_division=0)
    rc  = recall_score   (y, hat, zero_division=0)
    return 0 if pr+rc == 0 else (1+alpha)*pr*rc/(alpha*pr+rc)

# ───── Model ─────
def build(cfg):
    l2 = regularizers.l2(cfg["l2"])
    inp = layers.Input(shape=(cfg["window"], n_feat))

    if cfg["arch"] == "conv_lstm":
        x = inp
        for _ in range(cfg["conv_blocks"]):
            x = layers.Conv1D(cfg["filters"], cfg["kernel"], padding="causal",
                              activation=cfg["act"], kernel_regularizer=l2)(x)
            x = layers.Dropout(cfg["conv_drop"])(x)
        x = layers.LSTM(cfg["units"], dropout=0.0)(x)             # cuDNN fast
    else:  # lstm_conv
        x = layers.LSTM(cfg["units"], dropout=0.0,
                        return_sequences=True)(inp)
        x = layers.Conv1D(cfg["filters"], cfg["kernel"], padding="same",
                          activation=cfg["act"], kernel_regularizer=l2)(x)
        x = (layers.GlobalMaxPooling1D() if cfg["pool"]=="gmp"
             else layers.GlobalAveragePooling1D())(x)

    x = layers.Dense(cfg["dense"], activation=cfg["act"], kernel_regularizer=l2)(x)
    x = layers.Dropout(cfg["dropout"])(x)
    out = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inp, out)

    w0 = tf.constant(W0, dtype=tf.float32)
    w1 = tf.constant(W1, dtype=tf.float32)
    def loss(y_t, y_p):
        y_t = tf.cast(y_t, tf.float32)
        weights = tf.where(tf.equal(y_t, 1), w1, w0)
        return tf.reduce_mean(weights * keras.losses.binary_crossentropy(y_t, y_p))

    opt = (keras.optimizers.AdamW(cfg["lr"], weight_decay=cfg["l2"])
           if cfg["optim"] == "adamw"
           else keras.optimizers.Nadam(cfg["lr"]) if cfg["optim"] == "nadam"
           else keras.optimizers.Adam(cfg["lr"]))
    model.compile(opt, loss=loss)
    return model

# ───── Optuna objective ─────
def objective(trial: optuna.trial.Trial):
    cfg = dict(
        arch        = trial.suggest_categorical("arch", ["conv_lstm","lstm_conv"]),
        window      = trial.suggest_int("window", 12, 72, step=6),
        filters     = trial.suggest_categorical("filters",[32,48,64,96,128]),
        kernel      = trial.suggest_int("kernel", 2, 7),
        units       = trial.suggest_categorical("units",[32,64,96,128,192]),
        conv_drop   = trial.suggest_float("conv_drop", 0, .3),
        dense       = trial.suggest_categorical("dense",[32,64,128]),
        dropout     = trial.suggest_float("dropout", 0, .4),
        l2          = trial.suggest_float("l2", 1e-6, 1e-3, log=True),
        lr          = trial.suggest_float("lr", 5e-5, 3e-3, log=True),
        batch       = trial.suggest_categorical("batch",[64,128,256]),
        act         = trial.suggest_categorical("act", ["relu","elu"]),
        pool        = trial.suggest_categorical("pool",["gmp","gap"]),
        conv_blocks = trial.suggest_int("conv_blocks", 1, 2),
        optim       = trial.suggest_categorical("optim", ["adam","adamw","nadam"]),
        units_dummy = 0  # placeholder so cfg dict has same keys in both branches
    )
    Xtr, ytr = make_windows(X_tr_raw, y_tr_raw, cfg["window"])
    Xva, yva = make_windows(X_va_raw, y_va_raw, cfg["window"])
    if len(Xtr) < cfg["batch"] * 4:
        return float("inf")

    ds_tr = (tf.data.Dataset.from_tensor_slices((Xtr, ytr))
             .shuffle(10_000).batch(cfg["batch"]).prefetch(tf.data.AUTOTUNE))
    ds_va = (tf.data.Dataset.from_tensor_slices((Xva, yva))
             .batch(cfg["batch"]).prefetch(tf.data.AUTOTUNE))  # not shuffled – label order preserved

    tf.keras.backend.clear_session(); gc.collect()
    model = build(cfg)

    cb = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, verbose=0),
          keras.callbacks.ReduceLROnPlateau(patience=5, factor=.5, min_lr=1e-6, verbose=0),
          TFKerasPruningCallback(trial, "val_loss")]
    model.fit(ds_tr, validation_data=ds_va, epochs=100, callbacks=cb, verbose=0)

    prob  = model.predict(ds_va, verbose=0).ravel()
    y_hat = (prob >= .5).astype(int)
    score = wf1(yva, prob)

    pr = precision_score(yva, y_hat, zero_division=0)
    rc = recall_score   (yva, y_hat, zero_division=0)
    trial.set_user_attr("precision", pr); trial.set_user_attr("recall", rc)

    print(f"T{trial.number:02d} | Fα2={score:.3f} P={pr:.3f} R={rc:.3f} "
          f"{cfg['arch']} win={cfg['window']}")

    del model; tf.keras.backend.clear_session(); gc.collect()
    return -score

# ───── Search ─────
study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=SEED, multivariate=True),
    pruner =optuna.pruners.MedianPruner(n_startup_trials=5))
print(f"🚀  Starting {N_TRIALS}-trial search (big batches, cuDNN, XLA)…")
study.optimize(objective, n_trials=N_TRIALS,
               timeout=TIMEOUT, show_progress_bar=True, gc_after_trial=True)

best, ts = study.best_trial, datetime.utcnow().strftime("%Y%m%d_%H%M%S")
print(json.dumps({**best.params, "Fα2": -best.value,
                  "P": best.user_attrs["precision"],
                  "R": best.user_attrs["recall"]}, indent=2))

json.dump(best.params, open(f"best_params_cnn_lstm_{ts}.json","w"), indent=2)
study.trials_dataframe().to_csv(f"trials_cnn_lstm_{ts}.csv", index=False)
print(f"📜 Done – scaler saved → {SCALER_PKL}")


[I 2025-06-11 14:26:29,625] A new study created in memory with name: no-name-7c072783-b210-43ed-a4cc-e469a26c0a4b


🚀  Starting 100-trial search (big batches, cuDNN, XLA)…


  0%|          | 0/100 [00:00<?, ?it/s]

T00 | Fα2=0.595 P=0.521 R=0.640 lstm_conv win=60


  0%|          | 0/100 [02:29<?, ?it/s]

...

[I 2025-06-11 16:28:39,182] Trial 98 pruned. Trial was pruned at epoch 0.


Best trial: 94. Best value: -0.765957:  99%|█████████▉| 99/100 [2:02:42<00:49, 49.01s/it, 7331.66/10800 seconds]

[I 2025-06-11 16:29:11,961] Trial 99 pruned. Trial was pruned at epoch 0.


Best trial: 94. Best value: -0.765957: 100%|██████████| 100/100 [2:02:44<00:00, 73.64s/it, 7364.45/10800 seconds]

{
  "arch": "lstm_conv",
  "window": 66,
  "filters": 32,
  "kernel": 4,
  "units": 128,
  "conv_drop": 0.17402498680568057,
  "dense": 64,
  "dropout": 0.38389775614113647,
  "l2": 2.3729148086507226e-06,
  "lr": 0.0028800779177212033,
  "batch": 64,
  "act": "relu",
  "pool": "gap",
  "conv_blocks": 2,
  "optim": "nadam",
  "F\u03b12": 0.7659574468085105,
  "P": 0.5217391304347826,
  "R": 1.0
}
📜 Done – scaler saved → cnn_lstm_scaler.pkl


In [ ]:
[I 2025-06-11 14:26:29,625] A new study created in memory with name: no-name-7c072783-b210-43ed-a4cc-e469a26c0a4b
🚀  Starting 100-trial search (big batches, cuDNN, XLA)…
  0%|          | 0/100 [00:00<?, ?it/s]T00 | Fα2=0.595 P=0.521 R=0.640 lstm_conv win=60
  0%|          | 0/100 [02:29<?, ?it/s][I 2025-06-11 14:28:59,301] Trial 0 finished with value: -0.5948424068767908 and parameters: {'arch': 'lstm_conv', 'window': 60, 'filters': 128, 'kernel': 5, 'units': 96, 'conv_drop': 0.05454749016213018, 'dense': 128, 'dropout': 0.17277800745684632, 'l2': 7.4763120622522945e-06, 'lr': 0.000612261560602803, 'batch': 256, 'act': 'elu', 'pool': 'gap', 'conv_blocks': 2, 'optim': 'adamw'}. Best is trial 0 with value: -0.5948424068767908.
Best trial: 0. Best value: -0.594842:   1%|          | 1/100 [02:31<4:09:18, 151.10s/it, 151.10/10800 seconds]T01 | Fα2=0.391 P=0.522 R=0.348 lstm_conv win=72
Best trial: 0. Best value: -0.594842:   1%|          | 1/100 [06:24<4:09:18, 151.10s/it, 151.10/10800 seconds][I 2025-06-11 14:32:53,658] Trial 1 finished with value: -0.3912740775121838 and parameters: {'arch': 'lstm_conv', 'window': 72, 'filters': 32, 'kernel': 2, 'units': 96, 'conv_drop': 0.09351332282682329, 'dense': 64, 'dropout': 0.38783385110582347, 'l2': 0.00021154290797261214, 'lr': 0.0023417537373087377, 'batch': 256, 'act': 'elu', 'pool': 'gap', 'conv_blocks': 1, 'optim': 'adamw'}. Best is trial 0 with value: -0.5948424068767908.
Best trial: 0. Best value: -0.594842:   2%|▏         | 2/100 [06:25<5:26:48, 200.09s/it, 385.48/10800 seconds]T02 | Fα2=0.547 P=0.518 R=0.563 lstm_conv win=18
Best trial: 0. Best value: -0.594842:   2%|▏         | 2/100 [06:50<5:26:48, 200.09s/it, 385.48/10800 seconds][I 2025-06-11 14:33:20,153] Trial 2 finished with value: -0.5472773736976606 and parameters: {'arch': 'lstm_conv', 'window': 18, 'filters': 64, 'kernel': 2, 'units': 32, 'conv_drop': 0.10753971856328177, 'dense': 64, 'dropout': 0.13235920994105968, 'l2': 1.5512259126484753e-06, 'lr': 0.00017862556214123332, 'batch': 128, 'act': 'relu', 'pool': 'gap', 'conv_blocks': 2, 'optim': 'adamw'}. Best is trial 0 with value: -0.5948424068767908.
Best trial: 0. Best value: -0.594842:   3%|▎         | 3/100 [06:51<3:15:20, 120.83s/it, 411.99/10800 seconds]T03 | Fα2=0.527 P=0.533 R=0.524 conv_lstm win=12
Best trial: 0. Best value: -0.594842:   3%|▎         | 3/100 [07:34<3:15:20, 120.83s/it, 411.99/10800 seconds][I 2025-06-11 14:34:03,842] Trial 3 finished with value: -0.5269035532994923 and parameters: {'arch': 'conv_lstm', 'window': 12, 'filters': 64, 'kernel': 7, 'units': 96, 'conv_drop': 0.0869254358741304, 'dense': 64, 'dropout': 0.2533615026041694, 'l2': 0.00041151130495610907, 'lr': 0.001342828384653336, 'batch': 128, 'act': 'elu', 'pool': 'gmp', 'conv_blocks': 1, 'optim': 'nadam'}. Best is trial 0 with value: -0.5948424068767908.
Best trial: 0. Best value: -0.594842:   4%|▍         | 4/100 [07:35<2:24:36, 90.38s/it, 455.69/10800 seconds] T04 | Fα2=0.677 P=0.518 R=0.800 lstm_conv win=36
Best trial: 0. Best value: -0.594842:   4%|▍         | 4/100 [09:37<2:24:36, 90.38s/it, 455.69/10800 seconds][I 2025-06-11 14:36:07,021] Trial 4 finished with value: -0.677185721676151 and parameters: {'arch': 'lstm_conv', 'window': 36, 'filters': 96, 'kernel': 5, 'units': 96, 'conv_drop': 0.14917455176771563, 'dense': 32, 'dropout': 0.24382573359195875, 'l2': 3.2213437409123405e-05, 'lr': 6.173152563090738e-05, 'batch': 128, 'act': 'elu', 'pool': 'gmp', 'conv_blocks': 2, 'optim': 'adam'}. Best is trial 4 with value: -0.677185721676151.
Best trial: 4. Best value: -0.677186:   5%|▌         | 5/100 [11:40<2:41:49, 102.21s/it, 578.87/10800 seconds][I 2025-06-11 14:38:10,590] Trial 5 pruned. Trial was pruned at epoch 16.
Best trial: 4. Best value: -0.677186:   6%|▌         | 6/100 [11:55<2:51:31, 109.48s/it, 702.47/10800 seconds][I 2025-06-11 14:38:25,192] Trial 6 pruned. Trial was pruned at epoch 0.
Best trial: 4. Best value: -0.677186:   7%|▋         | 7/100 [11:57<2:01:36, 78.46s/it, 717.05/10800 seconds] T07 | Fα2=0.691 P=0.519 R=0.828 conv_lstm win=72
Best trial: 4. Best value: -0.677186:   7%|▋         | 7/100 [14:14<2:01:36, 78.46s/it, 717.05/10800 seconds][I 2025-06-11 14:40:44,050] Trial 7 finished with value: -0.6909967292132897 and parameters: {'arch': 'conv_lstm', 'window': 72, 'filters': 48, 'kernel': 5, 'units': 96, 'conv_drop': 0.19364168877215035, 'dense': 128, 'dropout': 0.3659457560881794, 'l2': 1.2896625857891445e-05, 'lr': 5.326650551390532e-05, 'batch': 256, 'act': 'relu', 'pool': 'gap', 'conv_blocks': 2, 'optim': 'nadam'}. Best is trial 7 with value: -0.6909967292132897.
Best trial: 7. Best value: -0.690997:   8%|▊         | 8/100 [14:39<2:29:51, 97.73s/it, 856.05/10800 seconds][I 2025-06-11 14:41:09,220] Trial 8 pruned. Trial was pruned at epoch 4.
Best trial: 7. Best value: -0.690997:   9%|▉         | 9/100 [14:52<1:53:49, 75.05s/it, 881.23/10800 seconds][I 2025-06-11 14:41:22,606] Trial 9 pruned. Trial was pruned at epoch 4.
Best trial: 7. Best value: -0.690997:  10%|█         | 10/100 [14:54<1:24:01, 56.01s/it, 894.61/10800 seconds]T10 | Fα2=0.685 P=0.532 R=0.801 conv_lstm win=72
Best trial: 7. Best value: -0.690997:  10%|█         | 10/100 [16:22<1:24:01, 56.01s/it, 894.61/10800 seconds][I 2025-06-11 14:42:51,866] Trial 10 finished with value: -0.6853813559322035 and parameters: {'arch': 'conv_lstm', 'window': 72, 'filters': 48, 'kernel': 3, 'units': 32, 'conv_drop': 0.14914175826424267, 'dense': 128, 'dropout': 0.36311711503988925, 'l2': 2.4631096227367697e-05, 'lr': 9.919404853103473e-05, 'batch': 64, 'act': 'relu', 'pool': 'gap', 'conv_blocks': 2, 'optim': 'nadam'}. Best is trial 7 with value: -0.6909967292132897.
Best trial: 7. Best value: -0.690997:  11%|█         | 11/100 [16:35<1:38:11, 66.19s/it, 983.89/10800 seconds][I 2025-06-11 14:43:04,653] Trial 11 pruned. Trial was pruned at epoch 0.
Best trial: 7. Best value: -0.690997:  12%|█▏        | 12/100 [16:36<1:13:17, 49.97s/it, 996.75/10800 seconds]T12 | Fα2=0.588 P=0.527 R=0.625 conv_lstm win=66
Best trial: 7. Best value: -0.690997:  12%|█▏        | 12/100 [18:20<1:13:17, 49.97s/it, 996.75/10800 seconds][I 2025-06-11 14:44:50,201] Trial 12 finished with value: -0.5884861407249468 and parameters: {'arch': 'conv_lstm', 'window': 66, 'filters': 48, 'kernel': 6, 'units': 96, 'conv_drop': 0.22606400599215065, 'dense': 128, 'dropout': 0.22176648138502159, 'l2': 4.5679942859048045e-06, 'lr': 5.5459545060680684e-05, 'batch': 128, 'act': 'relu', 'pool': 'gap', 'conv_blocks': 1, 'optim': 'nadam'}. Best is trial 7 with value: -0.6909967292132897.
Best trial: 7. Best value: -0.690997:  13%|█▎        | 13/100 [18:31<1:36:52, 66.82s/it, 1102.33/10800 seconds][I 2025-06-11 14:45:01,423] Trial 13 pruned. Trial was pruned at epoch 0.
Best trial: 7. Best value: -0.690997:  14%|█▍        | 14/100 [18:33<1:11:41, 50.01s/it, 1113.51/10800 seconds]T14 | Fα2=0.659 P=0.530 R=0.750 conv_lstm win=60
Best trial: 7. Best value: -0.690997:  14%|█▍        | 14/100 [20:00<1:11:41, 50.01s/it, 1113.51/10800 seconds][I 2025-06-11 14:46:29,946] Trial 14 finished with value: -0.6591442498645965 and parameters: {'arch': 'conv_lstm', 'window': 60, 'filters': 48, 'kernel': 7, 'units': 96, 'conv_drop': 0.16854365802480042, 'dense': 128, 'dropout': 0.34212635055038104, 'l2': 5.34107900020877e-06, 'lr': 8.139951880089569e-05, 'batch': 256, 'act': 'relu', 'pool': 'gap', 'conv_blocks': 2, 'optim': 'adam'}. Best is trial 7 with value: -0.6909967292132897.
Best trial: 7. Best value: -0.690997:  15%|█▌        | 15/100 [20:10<1:27:16, 61.60s/it, 1201.98/10800 seconds][I 2025-06-11 14:46:40,476] Trial 15 pruned. Trial was pruned at epoch 0.
Best trial: 7. Best value: -0.690997:  16%|█▌        | 16/100 [20:26<1:04:43, 46.24s/it, 1212.53/10800 seconds][I 2025-06-11 14:46:56,098] Trial 16 pruned. Trial was pruned at epoch 0.
Best trial: 7. Best value: -0.690997:  17%|█▋        | 17/100 [20:28<51:14, 37.04s/it, 1228.20/10800 seconds]  T17 | Fα2=0.709 P=0.519 R=0.868 conv_lstm win=72
Best trial: 7. Best value: -0.690997:  17%|█▋        | 17/100 [21:57<51:14, 37.04s/it, 1228.20/10800 seconds][I 2025-06-11 14:48:26,804] Trial 17 finished with value: -0.7087967644084935 and parameters: {'arch': 'conv_lstm', 'window': 72, 'filters': 48, 'kernel': 2, 'units': 32, 'conv_drop': 0.2020859487428539, 'dense': 128, 'dropout': 0.3311343605595314, 'l2': 5.446410711918106e-06, 'lr': 0.0002933566355803551, 'batch': 64, 'act': 'relu', 'pool': 'gmp', 'conv_blocks': 2, 'optim': 'nadam'}. Best is trial 17 with value: -0.7087967644084935.
Best trial: 17. Best value: -0.708797:  18%|█▊        | 18/100 [21:58<1:12:35, 53.11s/it, 1318.71/10800 seconds]T18 | Fα2=0.729 P=0.529 R=0.898 conv_lstm win=66
Best trial: 17. Best value: -0.708797:  18%|█▊        | 18/100 [26:04<1:12:35, 53.11s/it, 1318.71/10800 seconds][I 2025-06-11 14:52:34,085] Trial 18 finished with value: -0.7285785869383664 and parameters: {'arch': 'conv_lstm', 'window': 66, 'filters': 48, 'kernel': 2, 'units': 128, 'conv_drop': 0.1829073485357999, 'dense': 64, 'dropout': 0.3367267654373487, 'l2': 1.9949353471421903e-06, 'lr': 0.0010969018481585208, 'batch': 64, 'act': 'relu', 'pool': 'gmp', 'conv_blocks': 2, 'optim': 'nadam'}. Best is trial 18 with value: -0.7285785869383664.
Best trial: 18. Best value: -0.728579:  19%|█▉        | 19/100 [26:19<2:30:26, 111.43s/it, 1566.01/10800 seconds][I 2025-06-11 14:52:48,629] Trial 19 pruned. Trial was pruned at epoch 0.
Best trial: 18. Best value: -0.728579:  20%|██        | 20/100 [26:20<1:49:48, 82.35s/it, 1580.59/10800 seconds] T20 | Fα2=0.732 P=0.524 R=0.914 conv_lstm win=60
Best trial: 18. Best value: -0.728579:  20%|██        | 20/100 [30:17<1:49:48, 82.35s/it, 1580.59/10800 seconds][I 2025-06-11 14:56:46,797] Trial 20 finished with value: -0.7319723411261113 and parameters: {'arch': 'conv_lstm', 'window': 60, 'filters': 96, 'kernel': 2, 'units': 128, 'conv_drop': 0.23169116379282378, 'dense': 128, 'dropout': 0.22993558336668984, 'l2': 1.1980316309987873e-06, 'lr': 0.0010514052377099624, 'batch': 64, 'act': 'relu', 'pool': 'gap', 'conv_blocks': 2, 'optim': 'nadam'}. Best is trial 20 with value: -0.7319723411261113.
Best trial: 20. Best value: -0.731972:  21%|██        | 21/100 [30:18<2:50:00, 129.12s/it, 1818.73/10800 seconds]T21 | Fα2=0.597 P=0.533 R=0.636 conv_lstm win=48
Best trial: 20. Best value: -0.731972:  21%|██        | 21/100 [33:25<2:50:00, 129.12s/it, 1818.73/10800 seconds][I 2025-06-11 14:59:54,707] Trial 21 finished with value: -0.5973476840284452 and parameters: {'arch': 'conv_lstm', 'window': 48, 'filters': 96, 'kernel': 3, 'units': 128, 'conv_drop': 0.13417079120549671, 'dense': 128, 'dropout': 0.20486313559166633, 'l2': 1.3625466836519646e-06, 'lr': 0.0009418760303623328, 'batch': 64, 'act': 'relu', 'pool': 'gmp', 'conv_blocks': 2, 'optim': 'nadam'}. Best is trial 20 with value: -0.7319723411261113.
Best trial: 20. Best value: -0.731972:  22%|██▏       | 22/100 [33:26<3:10:46, 146.75s/it, 2006.62/10800 seconds]T22 | Fα2=0.672 P=0.528 R=0.778 conv_lstm win=66
Best trial: 20. Best value: -0.731972:  22%|██▏       | 22/100 [37:25<3:10:46, 146.75s/it, 2006.62/10800 seconds][I 2025-06-11 15:03:55,538] Trial 22 finished with value: -0.6717611515905456 and parameters: {'arch': 'conv_lstm', 'window': 66, 'filters': 64, 'kernel': 2, 'units': 128, 'conv_drop': 0.20725498441456727, 'dense': 64, 'dropout': 0.1514327494157322, 'l2': 3.4372389137912706e-06, 'lr': 0.0011623426402264853, 'batch': 64, 'act': 'relu', 'pool': 'gap', 'conv_blocks': 2, 'optim': 'nadam'}. Best is trial 20 with value: -0.7319723411261113.
Best trial: 20. Best value: -0.731972:  23%|██▎       | 23/100 [37:58<3:44:35, 175.01s/it, 2247.53/10800 seconds][I 2025-06-11 15:04:28,071] Trial 23 pruned. Trial was pruned at epoch 0.
Best trial: 20. Best value: -0.731972:  24%|██▍       | 24/100 [38:00<2:47:31, 132.26s/it, 2280.06/10800 seconds]T24 | Fα2=0.717 P=0.523 R=0.880 lstm_conv win=60
Best trial: 20. Best value: -0.731972:  24%|██▍       | 24/100 [40:47<2:47:31, 132.26s/it, 2280.06/10800 seconds][I 2025-06-11 15:07:16,928] Trial 24 finished with value: -0.7172275238573581 and parameters: {'arch': 'lstm_conv', 'window': 60, 'filters': 48, 'kernel': 3, 'units': 128, 'conv_drop': 0.22458452100672904, 'dense': 64, 'dropout': 0.27475311523806734, 'l2': 1.162732997626243e-06, 'lr': 0.0012399873702159075, 'batch': 256, 'act': 'relu', 'pool': 'gmp', 'conv_blocks': 2, 'optim': 'adamw'}. Best is trial 20 with value: -0.7319723411261113.
Best trial: 20. Best value: -0.731972:  25%|██▌       | 25/100 [41:06<2:59:02, 143.23s/it, 2448.89/10800 seconds][I 2025-06-11 15:07:35,721] Trial 25 pruned. Trial was pruned at epoch 0.
Best trial: 20. Best value: -0.731972:  26%|██▌       | 26/100 [41:07<2:10:37, 105.91s/it, 2467.72/10800 seconds]T26 | Fα2=0.708 P=0.524 R=0.859 lstm_conv win=66
Best trial: 20. Best value: -0.731972:  26%|██▌       | 26/100 [43:12<2:10:37, 105.91s/it, 2467.72/10800 seconds][I 2025-06-11 15:09:42,598] Trial 26 finished with value: -0.7082767978290366 and parameters: {'arch': 'lstm_conv', 'window': 66, 'filters': 48, 'kernel': 3, 'units': 64, 'conv_drop': 0.18808609956374508, 'dense': 128, 'dropout': 0.3124217042135821, 'l2': 1.701627483058166e-06, 'lr': 0.0020810000414228608, 'batch': 256, 'act': 'relu', 'pool': 'gmp', 'conv_blocks': 2, 'optim': 'adamw'}. Best is trial 20 with value: -0.7319723411261113.
Best trial: 20. Best value: -0.731972:  27%|██▋       | 27/100 [43:28<2:16:33, 112.24s/it, 2594.73/10800 seconds][I 2025-06-11 15:09:58,292] Trial 27 pruned. Trial was pruned at epoch 0.
Best trial: 20. Best value: -0.731972:  28%|██▊       | 28/100 [43:30<1:39:55, 83.27s/it, 2610.43/10800 seconds] T28 | Fα2=0.703 P=0.526 R=0.845 lstm_conv win=72
Best trial: 20. Best value: -0.731972:  28%|██▊       | 28/100 [47:08<1:39:55, 83.27s/it, 2610.43/10800 seconds][I 2025-06-11 15:13:38,257] Trial 28 finished with value: -0.703036541430777 and parameters: {'arch': 'lstm_conv', 'window': 72, 'filters': 128, 'kernel': 5, 'units': 128, 'conv_drop': 0.17053482179312465, 'dense': 64, 'dropout': 0.33831466588119824, 'l2': 1.6450611097511145e-06, 'lr': 0.0010824559227546788, 'batch': 64, 'act': 'relu', 'pool': 'gmp', 'conv_blocks': 2, 'optim': 'adamw'}. Best is trial 20 with value: -0.7319723411261113.
Best trial: 20. Best value: -0.731972:  29%|██▉       | 29/100 [47:10<2:27:05, 124.30s/it, 2830.44/10800 seconds]T29 | Fα2=0.692 P=0.521 R=0.829 conv_lstm win=60
Best trial: 20. Best value: -0.731972:  29%|██▉       | 29/100 [49:44<2:27:05, 124.30s/it, 2830.44/10800 seconds][I 2025-06-11 15:16:14,025] Trial 29 finished with value: -0.6923076923076923 and parameters: {'arch': 'conv_lstm', 'window': 60, 'filters': 32, 'kernel': 2, 'units': 128, 'conv_drop': 0.17655258017029987, 'dense': 64, 'dropout': 0.3750434027374682, 'l2': 1.5304402108416722e-06, 'lr': 0.0018149865587916117, 'batch': 64, 'act': 'relu', 'pool': 'gmp', 'conv_blocks': 1, 'optim': 'nadam'}. Best is trial 20 with value: -0.7319723411261113.
Best trial: 20. Best value: -0.731972:  30%|███       | 30/100 [49:58<2:36:01, 133.73s/it, 2986.19/10800 seconds][I 2025-06-11 15:16:28,624] Trial 30 pruned. Trial was pruned at epoch 0.
Best trial: 20. Best value: -0.731972:  31%|███       | 31/100 [50:11<1:52:41, 97.99s/it, 3000.77/10800 seconds] [I 2025-06-11 15:16:40,861] Trial 31 pruned. Trial was pruned at epoch 0.
Best trial: 20. Best value: -0.731972:  32%|███▏      | 32/100 [50:32<1:21:55, 72.29s/it, 3013.09/10800 seconds][I 2025-06-11 15:17:02,199] Trial 32 pruned. Trial was pruned at epoch 0.
Best trial: 20. Best value: -0.731972:  33%|███▎      | 33/100 [50:34<1:03:38, 57.00s/it, 3034.42/10800 seconds]T33 | Fα2=0.628 P=0.531 R=0.691 conv_lstm win=66
Best trial: 20. Best value: -0.731972:  33%|███▎      | 33/100 [51:37<1:03:38, 57.00s/it, 3034.42/10800 seconds][I 2025-06-11 15:18:07,065] Trial 33 finished with value: -0.628154795288839 and parameters: {'arch': 'conv_lstm', 'window': 66, 'filters': 64, 'kernel': 4, 'units': 32, 'conv_drop': 0.25301886959374464, 'dense': 128, 'dropout': 0.3266113420716731, 'l2': 2.565111517777266e-06, 'lr': 0.0004553361197772574, 'batch': 128, 'act': 'relu', 'pool': 'gmp', 'conv_blocks': 2, 'optim': 'nadam'}. Best is trial 20 with value: -0.7319723411261113.
Best trial: 20. Best value: -0.731972:  34%|███▍      | 34/100 [51:58<1:05:16, 59.34s/it, 3099.23/10800 seconds][I 2025-06-11 15:18:28,032] Trial 34 pruned. Trial was pruned at epoch 0.
Best trial: 20. Best value: -0.731972:  35%|███▌      | 35/100 [52:00<51:50, 47.85s/it, 3120.26/10800 seconds]  T35 | Fα2=0.602 P=0.539 R=0.639 conv_lstm win=66
Best trial: 20. Best value: -0.731972:  35%|███▌      | 35/100 [57:05<51:50, 47.85s/it, 3120.26/10800 seconds][I 2025-06-11 15:23:34,952] Trial 35 finished with value: -0.6018608257414227 and parameters: {'arch': 'conv_lstm', 'window': 66, 'filters': 48, 'kernel': 2, 'units': 128, 'conv_drop': 0.2290901400745524, 'dense': 128, 'dropout': 0.35586673619835707, 'l2': 3.4370979194197407e-06, 'lr': 0.0002580404573248242, 'batch': 64, 'act': 'relu', 'pool': 'gmp', 'conv_blocks': 2, 'optim': 'nadam'}. Best is trial 20 with value: -0.7319723411261113.
Best trial: 20. Best value: -0.731972:  36%|███▌      | 36/100 [57:29<2:13:53, 125.52s/it, 3427.00/10800 seconds][I 2025-06-11 15:23:58,720] Trial 36 pruned. Trial was pruned at epoch 0.
Best trial: 20. Best value: -0.731972:  37%|███▋      | 37/100 [57:50<1:39:44, 95.00s/it, 3450.79/10800 seconds] [I 2025-06-11 15:24:20,498] Trial 37 pruned. Trial was pruned at epoch 0.
Best trial: 20. Best value: -0.731972:  38%|███▊      | 38/100 [58:02<1:15:28, 73.04s/it, 3472.58/10800 seconds][I 2025-06-11 15:24:32,309] Trial 38 pruned. Trial was pruned at epoch 0.
Best trial: 20. Best value: -0.731972:  39%|███▉      | 39/100 [58:16<55:34, 54.67s/it, 3484.39/10800 seconds]  [I 2025-06-11 15:24:45,997] Trial 39 pruned. Trial was pruned at epoch 0.
Best trial: 20. Best value: -0.731972:  40%|████      | 40/100 [58:18<42:23, 42.39s/it, 3498.12/10800 seconds]T40 | Fα2=0.636 P=0.530 R=0.706 conv_lstm win=72
Best trial: 20. Best value: -0.731972:  40%|████      | 40/100 [59:58<42:23, 42.39s/it, 3498.12/10800 seconds][I 2025-06-11 15:26:27,891] Trial 40 finished with value: -0.6355365763089491 and parameters: {'arch': 'conv_lstm', 'window': 72, 'filters': 48, 'kernel': 3, 'units': 32, 'conv_drop': 0.2991414066265154, 'dense': 128, 'dropout': 0.3298017348264688, 'l2': 7.605803562468741e-06, 'lr': 0.00012155054083183725, 'batch': 256, 'act': 'relu', 'pool': 'gmp', 'conv_blocks': 2, 'optim': 'adam'}. Best is trial 20 with value: -0.7319723411261113.
Best trial: 20. Best value: -0.731972:  41%|████      | 41/100 [1:00:09<59:17, 60.30s/it, 3600.21/10800 seconds][I 2025-06-11 15:26:39,012] Trial 41 pruned. Trial was pruned at epoch 0.
Best trial: 20. Best value: -0.731972:  42%|████▏     | 42/100 [1:00:26<44:00, 45.52s/it, 3611.26/10800 seconds][I 2025-06-11 15:26:56,197] Trial 42 pruned. Trial was pruned at epoch 0.
Best trial: 20. Best value: -0.731972:  43%|████▎     | 43/100 [1:00:28<35:09, 37.01s/it, 3628.42/10800 seconds]T43 | Fα2=0.763 P=0.521 R=0.994 lstm_conv win=66
Best trial: 20. Best value: -0.731972:  43%|████▎     | 43/100 [1:02:17<35:09, 37.01s/it, 3628.42/10800 seconds][I 2025-06-11 15:28:47,316] Trial 43 finished with value: -0.7631537367672618 and parameters: {'arch': 'lstm_conv', 'window': 66, 'filters': 32, 'kernel': 4, 'units': 64, 'conv_drop': 0.09979414734158484, 'dense': 128, 'dropout': 0.3879128314126981, 'l2': 2.073472158914153e-06, 'lr': 0.0025838646128367514, 'batch': 256, 'act': 'relu', 'pool': 'gmp', 'conv_blocks': 2, 'optim': 'adamw'}. Best is trial 43 with value: -0.7631537367672618.
Best trial: 43. Best value: -0.763154:  44%|████▍     | 44/100 [1:02:32<55:15, 59.21s/it, 3739.41/10800 seconds][I 2025-06-11 15:29:02,165] Trial 44 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  45%|████▌     | 45/100 [1:02:56<42:06, 45.94s/it, 3754.40/10800 seconds][I 2025-06-11 15:29:26,136] Trial 45 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  46%|████▌     | 46/100 [1:03:08<35:26, 39.38s/it, 3778.46/10800 seconds][I 2025-06-11 15:29:37,875] Trial 46 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  47%|████▋     | 47/100 [1:03:18<27:27, 31.08s/it, 3790.18/10800 seconds][I 2025-06-11 15:29:47,671] Trial 47 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  48%|████▊     | 48/100 [1:03:19<21:24, 24.70s/it, 3799.98/10800 seconds]T48 | Fα2=0.555 P=0.524 R=0.572 lstm_conv win=60
Best trial: 43. Best value: -0.763154:  48%|████▊     | 48/100 [1:04:38<21:24, 24.70s/it, 3799.98/10800 seconds][I 2025-06-11 15:31:08,037] Trial 48 finished with value: -0.555245313123255 and parameters: {'arch': 'lstm_conv', 'window': 60, 'filters': 96, 'kernel': 3, 'units': 32, 'conv_drop': 0.17899377038959471, 'dense': 128, 'dropout': 0.22152697641509517, 'l2': 1.3640561599870486e-06, 'lr': 0.00042314293757819894, 'batch': 64, 'act': 'relu', 'pool': 'gap', 'conv_blocks': 2, 'optim': 'nadam'}. Best is trial 43 with value: -0.7631537367672618.
Best trial: 43. Best value: -0.763154:  49%|████▉     | 49/100 [1:04:54<35:12, 41.41s/it, 3880.40/10800 seconds][I 2025-06-11 15:31:24,574] Trial 49 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  50%|█████     | 50/100 [1:05:27<28:14, 33.89s/it, 3896.73/10800 seconds][I 2025-06-11 15:31:57,236] Trial 50 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  51%|█████     | 51/100 [1:05:43<27:24, 33.56s/it, 3929.52/10800 seconds][I 2025-06-11 15:32:13,485] Trial 51 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  52%|█████▏    | 52/100 [1:06:03<22:41, 28.37s/it, 3945.78/10800 seconds][I 2025-06-11 15:32:33,364] Trial 52 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  53%|█████▎    | 53/100 [1:06:05<20:13, 25.82s/it, 3965.64/10800 seconds]T53 | Fα2=0.753 P=0.524 R=0.963 conv_lstm win=42
Best trial: 43. Best value: -0.763154:  53%|█████▎    | 53/100 [1:07:20<20:13, 25.82s/it, 3965.64/10800 seconds][I 2025-06-11 15:33:49,953] Trial 53 finished with value: -0.7526332588573252 and parameters: {'arch': 'conv_lstm', 'window': 42, 'filters': 48, 'kernel': 4, 'units': 64, 'conv_drop': 0.1852717446722201, 'dense': 128, 'dropout': 0.38016448336315367, 'l2': 1.5211958873031164e-06, 'lr': 0.001880948993648861, 'batch': 256, 'act': 'relu', 'pool': 'gmp', 'conv_blocks': 2, 'optim': 'adamw'}. Best is trial 43 with value: -0.7631537367672618.
Best trial: 43. Best value: -0.763154:  54%|█████▍    | 54/100 [1:07:38<31:26, 41.02s/it, 4042.12/10800 seconds][I 2025-06-11 15:34:08,476] Trial 54 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  55%|█████▌    | 55/100 [1:07:52<25:42, 34.28s/it, 4060.69/10800 seconds][I 2025-06-11 15:34:21,987] Trial 55 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  56%|█████▌    | 56/100 [1:07:54<20:35, 28.08s/it, 4074.31/10800 seconds]T56 | Fα2=0.754 P=0.523 R=0.968 conv_lstm win=72
Best trial: 43. Best value: -0.763154:  56%|█████▌    | 56/100 [1:11:23<20:35, 28.08s/it, 4074.31/10800 seconds][I 2025-06-11 15:37:53,323] Trial 56 finished with value: -0.7543408360128617 and parameters: {'arch': 'conv_lstm', 'window': 72, 'filters': 48, 'kernel': 4, 'units': 128, 'conv_drop': 0.13797735437663072, 'dense': 64, 'dropout': 0.23075557455386583, 'l2': 1.49581959499061e-06, 'lr': 0.001599498160491714, 'batch': 64, 'act': 'relu', 'pool': 'gmp', 'conv_blocks': 2, 'optim': 'nadam'}. Best is trial 43 with value: -0.7631537367672618.
Best trial: 43. Best value: -0.763154:  57%|█████▋    | 57/100 [1:11:49<59:32, 83.07s/it, 4285.70/10800 seconds][I 2025-06-11 15:38:19,342] Trial 57 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  58%|█████▊    | 58/100 [1:12:15<46:10, 65.96s/it, 4311.72/10800 seconds][I 2025-06-11 15:38:45,117] Trial 58 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  59%|█████▉    | 59/100 [1:12:27<36:49, 53.90s/it, 4337.47/10800 seconds][I 2025-06-11 15:38:57,152] Trial 59 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  60%|██████    | 60/100 [1:12:29<27:33, 41.34s/it, 4349.51/10800 seconds]T60 | Fα2=0.756 P=0.521 R=0.978 lstm_conv win=60
Best trial: 43. Best value: -0.763154:  60%|██████    | 60/100 [1:16:00<27:33, 41.34s/it, 4349.51/10800 seconds][I 2025-06-11 15:42:29,909] Trial 60 finished with value: -0.7564387917329093 and parameters: {'arch': 'lstm_conv', 'window': 60, 'filters': 48, 'kernel': 2, 'units': 128, 'conv_drop': 0.1786845807647062, 'dense': 64, 'dropout': 0.359900026827392, 'l2': 3.3000499138330928e-06, 'lr': 0.00193590456968632, 'batch': 64, 'act': 'relu', 'pool': 'gap', 'conv_blocks': 2, 'optim': 'nadam'}. Best is trial 43 with value: -0.7631537367672618.
Best trial: 43. Best value: -0.763154:  61%|██████    | 61/100 [1:16:16<1:00:15, 92.71s/it, 4562.09/10800 seconds][I 2025-06-11 15:42:45,953] Trial 61 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  62%|██████▏   | 62/100 [1:16:29<44:11, 69.78s/it, 4578.37/10800 seconds]  [I 2025-06-11 15:42:58,972] Trial 62 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  63%|██████▎   | 63/100 [1:16:40<32:31, 52.76s/it, 4591.40/10800 seconds][I 2025-06-11 15:43:10,621] Trial 63 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  64%|██████▍   | 64/100 [1:17:09<24:15, 40.43s/it, 4603.07/10800 seconds][I 2025-06-11 15:43:38,816] Trial 64 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  65%|██████▌   | 65/100 [1:17:11<21:26, 36.76s/it, 4631.27/10800 seconds]T65 | Fα2=0.738 P=0.521 R=0.932 lstm_conv win=72
Best trial: 43. Best value: -0.763154:  65%|██████▌   | 65/100 [1:23:29<21:26, 36.76s/it, 4631.27/10800 seconds][I 2025-06-11 15:49:59,618] Trial 65 finished with value: -0.7381146871426237 and parameters: {'arch': 'lstm_conv', 'window': 72, 'filters': 48, 'kernel': 3, 'units': 128, 'conv_drop': 0.2616057646267626, 'dense': 64, 'dropout': 0.2801565061718615, 'l2': 2.814305011848096e-06, 'lr': 0.0009199291392622619, 'batch': 64, 'act': 'relu', 'pool': 'gap', 'conv_blocks': 2, 'optim': 'nadam'}. Best is trial 43 with value: -0.7631537367672618.
Best trial: 43. Best value: -0.763154:  66%|██████▌   | 66/100 [1:24:20<1:19:17, 139.92s/it, 5011.88/10800 seconds][I 2025-06-11 15:50:50,310] Trial 66 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  67%|██████▋   | 67/100 [1:25:02<1:02:14, 113.17s/it, 5062.66/10800 seconds][I 2025-06-11 15:51:32,298] Trial 67 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  68%|██████▊   | 68/100 [1:25:33<48:57, 91.79s/it, 5104.55/10800 seconds]   [I 2025-06-11 15:52:03,261] Trial 68 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  69%|██████▉   | 69/100 [1:26:22<37:59, 73.54s/it, 5135.51/10800 seconds][I 2025-06-11 15:52:52,527] Trial 69 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  70%|███████   | 70/100 [1:26:24<33:07, 66.27s/it, 5184.80/10800 seconds]T70 | Fα2=0.605 P=0.524 R=0.655 conv_lstm win=72
Best trial: 43. Best value: -0.763154:  70%|███████   | 70/100 [1:28:54<33:07, 66.27s/it, 5184.80/10800 seconds][I 2025-06-11 15:55:23,937] Trial 70 finished with value: -0.604912414318355 and parameters: {'arch': 'conv_lstm', 'window': 72, 'filters': 48, 'kernel': 4, 'units': 64, 'conv_drop': 0.2676245859513692, 'dense': 32, 'dropout': 0.23038576915445977, 'l2': 2.0192902389379724e-06, 'lr': 0.0004206138733297379, 'batch': 64, 'act': 'relu', 'pool': 'gap', 'conv_blocks': 2, 'optim': 'nadam'}. Best is trial 43 with value: -0.7631537367672618.
Best trial: 43. Best value: -0.763154:  71%|███████   | 71/100 [1:29:22<44:22, 91.80s/it, 5336.19/10800 seconds][I 2025-06-11 15:55:52,158] Trial 71 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  72%|███████▏  | 72/100 [1:29:38<33:56, 72.74s/it, 5364.43/10800 seconds][I 2025-06-11 15:56:08,311] Trial 72 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  73%|███████▎  | 73/100 [1:29:55<25:05, 55.76s/it, 5380.60/10800 seconds][I 2025-06-11 15:56:24,930] Trial 73 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  74%|███████▍  | 74/100 [1:30:32<19:04, 44.02s/it, 5397.23/10800 seconds][I 2025-06-11 15:57:02,346] Trial 74 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  75%|███████▌  | 75/100 [1:31:14<17:31, 42.04s/it, 5434.65/10800 seconds][I 2025-06-11 15:57:43,718] Trial 75 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  76%|███████▌  | 76/100 [1:31:49<16:44, 41.84s/it, 5476.03/10800 seconds][I 2025-06-11 15:58:18,996] Trial 76 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  77%|███████▋  | 77/100 [1:32:13<15:17, 39.87s/it, 5511.31/10800 seconds][I 2025-06-11 15:58:42,773] Trial 77 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  78%|███████▊  | 78/100 [1:32:49<12:51, 35.05s/it, 5535.09/10800 seconds][I 2025-06-11 15:59:19,229] Trial 78 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  79%|███████▉  | 79/100 [1:33:26<12:24, 35.47s/it, 5571.56/10800 seconds][I 2025-06-11 15:59:56,466] Trial 79 pruned. Trial was pruned at epoch 0.
Best trial: 43. Best value: -0.763154:  80%|████████  | 80/100 [1:33:28<12:00, 36.00s/it, 5608.80/10800 seconds]T80 | Fα2=0.766 P=0.522 R=1.000 conv_lstm win=48
Best trial: 43. Best value: -0.763154:  80%|████████  | 80/100 [1:38:31<12:00, 36.00s/it, 5608.80/10800 seconds][I 2025-06-11 16:05:01,249] Trial 80 finished with value: -0.765867418899859 and parameters: {'arch': 'conv_lstm', 'window': 48, 'filters': 96, 'kernel': 4, 'units': 128, 'conv_drop': 0.25353715468406557, 'dense': 128, 'dropout': 0.2400109393045205, 'l2': 1.2330234316739454e-06, 'lr': 0.0027549451673990243, 'batch': 64, 'act': 'relu', 'pool': 'gap', 'conv_blocks': 2, 'optim': 'nadam'}. Best is trial 80 with value: -0.765867418899859.
Best trial: 80. Best value: -0.765867:  81%|████████  | 81/100 [1:38:33<36:56, 116.64s/it, 5913.58/10800 seconds]T81 | Fα2=0.754 P=0.522 R=0.970 conv_lstm win=54
Best trial: 80. Best value: -0.765867:  81%|████████  | 81/100 [1:40:18<36:56, 116.64s/it, 5913.58/10800 seconds][I 2025-06-11 16:06:48,279] Trial 81 finished with value: -0.7538265306122449 and parameters: {'arch': 'conv_lstm', 'window': 54, 'filters': 32, 'kernel': 6, 'units': 32, 'conv_drop': 0.24780265564248816, 'dense': 128, 'dropout': 0.15203249635957786, 'l2': 1.4605959616680251e-06, 'lr': 0.0014485959437495195, 'batch': 64, 'act': 'relu', 'pool': 'gap', 'conv_blocks': 2, 'optim': 'nadam'}. Best is trial 80 with value: -0.765867418899859.
Best trial: 80. Best value: -0.765867:  82%|████████▏ | 82/100 [1:40:20<34:07, 113.75s/it, 6020.61/10800 seconds]T82 | Fα2=0.745 P=0.525 R=0.943 conv_lstm win=60
Best trial: 80. Best value: -0.765867:  82%|████████▏ | 82/100 [1:48:08<34:07, 113.75s/it, 6020.61/10800 seconds][I 2025-06-11 16:14:37,891] Trial 82 finished with value: -0.7454929348708786 and parameters: {'arch': 'conv_lstm', 'window': 60, 'filters': 32, 'kernel': 5, 'units': 192, 'conv_drop': 0.28405085280328335, 'dense': 128, 'dropout': 0.07610006940561165, 'l2': 2.0952605078380426e-06, 'lr': 0.002044950575846103, 'batch': 64, 'act': 'relu', 'pool': 'gap', 'conv_blocks': 2, 'optim': 'nadam'}. Best is trial 80 with value: -0.765867418899859.
Best trial: 80. Best value: -0.765867:  83%|████████▎ | 83/100 [1:48:10<1:02:28, 220.52s/it, 6490.24/10800 seconds]T83 | Fα2=0.758 P=0.522 R=0.978 conv_lstm win=48
Best trial: 80. Best value: -0.765867:  83%|████████▎ | 83/100 [1:49:33<1:02:28, 220.52s/it, 6490.24/10800 seconds][I 2025-06-11 16:16:03,581] Trial 83 finished with value: -0.7576093849080532 and parameters: {'arch': 'conv_lstm', 'window': 48, 'filters': 32, 'kernel': 5, 'units': 32, 'conv_drop': 0.2021224153022909, 'dense': 128, 'dropout': 0.13554867341584192, 'l2': 3.0130866982184018e-06, 'lr': 0.0008022301262628734, 'batch': 64, 'act': 'relu', 'pool': 'gap', 'conv_blocks': 2, 'optim': 'nadam'}. Best is trial 80 with value: -0.765867418899859.
Best trial: 80. Best value: -0.765867:  84%|████████▍ | 84/100 [1:49:35<48:01, 180.07s/it, 6575.93/10800 seconds]  T84 | Fα2=0.686 P=0.525 R=0.811 conv_lstm win=48
Best trial: 80. Best value: -0.765867:  84%|████████▍ | 84/100 [1:51:35<48:01, 180.07s/it, 6575.93/10800 seconds][I 2025-06-11 16:18:04,923] Trial 84 finished with value: -0.686471505283215 and parameters: {'arch': 'conv_lstm', 'window': 48, 'filters': 32, 'kernel': 6, 'units': 64, 'conv_drop': 0.16140691771117316, 'dense': 128, 'dropout': 0.12281630692639198, 'l2': 2.706615571231907e-06, 'lr': 0.0011431980447298225, 'batch': 64, 'act': 'relu', 'pool': 'gap', 'conv_blocks': 2, 'optim': 'nadam'}. Best is trial 80 with value: -0.765867418899859.
Best trial: 80. Best value: -0.765867:  85%|████████▌ | 85/100 [1:52:25<40:36, 162.45s/it, 6697.28/10800 seconds][I 2025-06-11 16:18:55,338] Trial 85 pruned. Trial was pruned at epoch 0.
Best trial: 80. Best value: -0.765867:  86%|████████▌ | 86/100 [1:52:36<30:05, 128.94s/it, 6748.03/10800 seconds][I 2025-06-11 16:19:06,526] Trial 86 pruned. Trial was pruned at epoch 0.
Best trial: 80. Best value: -0.765867:  87%|████████▋ | 87/100 [1:52:50<20:16, 93.60s/it, 6759.16/10800 seconds] [I 2025-06-11 16:19:19,794] Trial 87 pruned. Trial was pruned at epoch 0.
Best trial: 80. Best value: -0.765867:  88%|████████▊ | 88/100 [1:53:04<13:53, 69.49s/it, 6772.39/10800 seconds][I 2025-06-11 16:19:34,223] Trial 88 pruned. Trial was pruned at epoch 0.
Best trial: 80. Best value: -0.765867:  89%|████████▉ | 89/100 [1:53:28<09:42, 52.97s/it, 6786.82/10800 seconds][I 2025-06-11 16:19:58,066] Trial 89 pruned. Trial was pruned at epoch 0.
Best trial: 80. Best value: -0.765867:  90%|█████████ | 90/100 [1:53:40<07:22, 44.24s/it, 6810.68/10800 seconds][I 2025-06-11 16:20:09,873] Trial 90 pruned. Trial was pruned at epoch 0.
Best trial: 80. Best value: -0.765867:  91%|█████████ | 91/100 [1:54:04<05:10, 34.51s/it, 6822.49/10800 seconds][I 2025-06-11 16:20:33,917] Trial 91 pruned. Trial was pruned at epoch 0.
Best trial: 80. Best value: -0.765867:  92%|█████████▏| 92/100 [1:54:16<04:11, 31.38s/it, 6846.55/10800 seconds][I 2025-06-11 16:20:46,310] Trial 92 pruned. Trial was pruned at epoch 0.
Best trial: 80. Best value: -0.765867:  93%|█████████▎| 93/100 [1:54:38<02:59, 25.69s/it, 6858.96/10800 seconds][I 2025-06-11 16:21:08,391] Trial 93 pruned. Trial was pruned at epoch 0.
Best trial: 80. Best value: -0.765867:  94%|█████████▍| 94/100 [1:54:41<02:27, 24.60s/it, 6881.04/10800 seconds]T94 | Fα2=0.766 P=0.522 R=1.000 lstm_conv win=66
Best trial: 80. Best value: -0.765867:  94%|█████████▍| 94/100 [2:00:14<02:27, 24.60s/it, 6881.04/10800 seconds][I 2025-06-11 16:26:44,439] Trial 94 finished with value: -0.7659574468085105 and parameters: {'arch': 'lstm_conv', 'window': 66, 'filters': 32, 'kernel': 4, 'units': 128, 'conv_drop': 0.17402498680568057, 'dense': 64, 'dropout': 0.38389775614113647, 'l2': 2.3729148086507226e-06, 'lr': 0.0028800779177212033, 'batch': 64, 'act': 'relu', 'pool': 'gap', 'conv_blocks': 2, 'optim': 'nadam'}. Best is trial 94 with value: -0.7659574468085105.
Best trial: 94. Best value: -0.765957:  95%|█████████▌| 95/100 [2:00:41<09:49, 117.98s/it, 7216.89/10800 seconds][I 2025-06-11 16:27:10,685] Trial 95 pruned. Trial was pruned at epoch 0.
Best trial: 94. Best value: -0.765957:  96%|█████████▌| 96/100 [2:01:13<06:01, 90.46s/it, 7243.16/10800 seconds] [I 2025-06-11 16:27:43,134] Trial 96 pruned. Trial was pruned at epoch 0.
Best trial: 94. Best value: -0.765957:  97%|█████████▋| 97/100 [2:01:53<03:39, 73.06s/it, 7275.62/10800 seconds][I 2025-06-11 16:28:23,150] Trial 97 pruned. Trial was pruned at epoch 0.
Best trial: 94. Best value: -0.765957:  98%|█████████▊| 98/100 [2:02:09<02:06, 63.15s/it, 7315.63/10800 seconds][I 2025-06-11 16:28:39,182] Trial 98 pruned. Trial was pruned at epoch 0.
Best trial: 94. Best value: -0.765957:  99%|█████████▉| 99/100 [2:02:42<00:49, 49.01s/it, 7331.66/10800 seconds][I 2025-06-11 16:29:11,961] Trial 99 pruned. Trial was pruned at epoch 0.
Best trial: 94. Best value: -0.765957: 100%|██████████| 100/100 [2:02:44<00:00, 73.64s/it, 7364.45/10800 seconds]{
  "arch": "lstm_conv",
  "window": 66,
  "filters": 32,
  "kernel": 4,
  "units": 128,
  "conv_drop": 0.17402498680568057,
  "dense": 64,
  "dropout": 0.38389775614113647,
  "l2": 2.3729148086507226e-06,
  "lr": 0.0028800779177212033,
  "batch": 64,
  "act": "relu",
  "pool": "gap",
  "conv_blocks": 2,
  "optim": "nadam",
  "F\u03b12": 0.7659574468085105,
  "P": 0.5217391304347826,
  "R": 1.0
}
📜 Done – scaler saved → cnn_lstm_scaler.pkl

In [ ]:


# ─── imports ───────────────────────────────────────────────────
import os, json, gc, warnings, joblib, numpy as np, pandas as pd
from datetime import datetime
from pathlib import Path
from typing import Dict, Tuple

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (classification_report, confusion_matrix,
                             precision_score, recall_score, f1_score,
                             accuracy_score, roc_curve, auc,
                             precision_recall_curve)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.callbacks import (ModelCheckpoint, EarlyStopping,
                                        ReduceLROnPlateau)

import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# ─── fixed params from Optuna ───────────────────────────────────────────
BEST_PARAMS = dict(
    arch="lstm_conv", window=24, filters=64, kernel=5, units=32,
    conv_drop=0.220, lstm_drop=0.285, dense=32, dropout=0.378,
    l2=1.449e-6, lr=0.00135, batch=32, act="relu", pool="gmp"
)

# ─── paths & config ────────────────────────────────────────────
CSV_PATH = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction"
                r"\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")
DROP_COLS = ["open","high","low","close","typical_price",
             "EMA_21","SMA_20","vwap_24h","close_4h"]

SEED, VAL_FRAC, TEST_FRAC, ALPHA = 42, .15, .15, 2.0
EPOCHS, PATIENCE = 200, 20

OUT_DIR = Path("model_outputs")
OUT_DIR.mkdir(exist_ok=True)
STAMP = datetime.now().strftime("%Y%m%d_%H%M%S")

np.random.seed(SEED)
tf.random.set_seed(SEED)

# GPU setup with feedback
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    for g in gpus:
        tf.config.experimental.set_memory_growth(g, True)
    print(f"✅ Found {len(gpus)} GPU(s)")
else:
    print("⚠️ No GPU found, using CPU")

# ─── data load & split ─────────────────────────────────────────────────
print("📊 Loading data...")
df = pd.read_csv(CSV_PATH, index_col=0, parse_dates=True)
df = df.loc["2018-01-01":]
df = df.drop(columns=[c for c in DROP_COLS if c in df.columns], errors='ignore')
df = df.dropna(subset=["target"]).dropna()

# Store feature names before dropping target
feat_names = [col for col in df.columns if col != "target"]

X_raw = df.drop(columns="target").values.astype(np.float32)
y_raw = df["target"].astype(np.int8).values
n_feat = X_raw.shape[1]

n = len(df)
val_start = int(n * (1 - VAL_FRAC - TEST_FRAC))
test_start = int(n * (1 - TEST_FRAC))
train_idx = np.arange(0, val_start)
val_idx = np.arange(val_start, test_start)
test_idx = np.arange(test_start, n)

scaler = StandardScaler().fit(X_raw[train_idx])
X_train = scaler.transform(X_raw[train_idx])
y_train = y_raw[train_idx]
X_val = scaler.transform(X_raw[val_idx])
y_val = y_raw[val_idx]
X_test = scaler.transform(X_raw[test_idx])
y_test = y_raw[test_idx]

joblib.dump(scaler, OUT_DIR / f"scaler_{STAMP}.pkl")

pos = y_train.mean()
W0, W1 = 1.0, (1-pos)/pos if pos else 1.0

print(f"Data shape: {df.shape}")
print(f"Train: {len(train_idx)}, Val: {len(val_idx)}, Test: {len(test_idx)}")
print(f"Positive rate: {pos:.3f}, Weights: W0={W0:.2f}, W1={W1:.2f}")

# ─── window helper ─────────────────────────────────────────────────────
def windows(X, y, w):
    xs, ys = [], []
    for i in range(w, len(X)):
        xs.append(X[i-w:i])
        ys.append(y[i])
    return np.asarray(xs, np.float32), np.asarray(ys, np.int8)

win = BEST_PARAMS["window"]
X_tr_w, y_tr_w = windows(X_train, y_train, win)
X_va_w, y_va_w = windows(X_val, y_val, win)
X_te_w, y_te_w = windows(X_test, y_test, win)

print(f"Window size: {win}, Train windows: {len(X_tr_w)}")

# ─── weighted F1 score ─────────────────────────────────────────────────
def wf1(y, proba, alpha=ALPHA):
    preds = (proba >= 0.5).astype(int)
    pr = precision_score(y, preds, zero_division=0)
    rc = recall_score(y, preds, zero_division=0)
    return 0 if pr + rc == 0 else (1 + alpha) * pr * rc / (alpha * pr + rc)

# ─── model builder ─────────────────────────────────────────────
def build(cfg):
    l2 = regularizers.l2(cfg["l2"])
    inp = layers.Input(shape=(cfg["window"], n_feat))

    # LSTM → Conv1D branch
    x = layers.LSTM(cfg["units"], dropout=cfg["lstm_drop"],
                    return_sequences=True,
                    kernel_regularizer=l2)(inp)
    x = layers.Conv1D(cfg["filters"], cfg["kernel"], padding="same",
                      activation=cfg["act"], kernel_regularizer=l2)(x)
    x = (layers.GlobalMaxPooling1D()(x) if cfg["pool"]=="gmp"
         else layers.GlobalAveragePooling1D()(x))
    x = layers.Dropout(cfg["conv_drop"])(x)

    x = layers.Dense(cfg["dense"], activation=cfg["act"],
                     kernel_regularizer=l2)(x)
    x = layers.Dropout(cfg["dropout"])(x)
    out = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inp, out)

    w0 = tf.constant(W0, dtype=tf.float32)
    w1 = tf.constant(W1, dtype=tf.float32)
    
    def wbce(y_t, y_p):
        y_t = tf.cast(y_t, tf.float32)
        w = tf.where(tf.equal(y_t, 1), w1, w0)
        return tf.reduce_mean(w * keras.losses.binary_crossentropy(y_t, y_p))

    model.compile(keras.optimizers.Adam(cfg["lr"]), loss=wbce,
                  metrics=['accuracy',
                           keras.metrics.Precision(name='precision'),
                           keras.metrics.Recall(name='recall'),
                           keras.metrics.AUC(name='auc')])
    return model

print("\n🏗️ Building model...")
model = build(BEST_PARAMS)
model.summary()

# ─── callbacks & training ──────────────────────────────────────────────
print("\n🚀 Starting training...")
ckpt_path = OUT_DIR / f"best_model_{STAMP}.keras"

history = model.fit(
    X_tr_w, y_tr_w,
    validation_data=(X_va_w, y_va_w),
    epochs=EPOCHS, batch_size=BEST_PARAMS["batch"],
    class_weight={0: W0, 1: W1},
    callbacks=[
        ModelCheckpoint(str(ckpt_path), save_best_only=True, monitor='val_loss', verbose=1),
        EarlyStopping(patience=PATIENCE, restore_best_weights=True, verbose=1),
        ReduceLROnPlateau(patience=10, factor=.5, min_lr=1e-6, verbose=1)
    ],
    verbose=1
)

model.save(OUT_DIR / f"final_model_{STAMP}.keras")

# ─── evaluation helper ─────────────────────────────────────────────────
def evaluate(X, y, name):
    proba = model.predict(X, verbose=0).ravel()
    preds = (proba >= 0.5).astype(int)
    pr = precision_score(y, preds, zero_division=0)
    rc = recall_score(y, preds, zero_division=0)
    f1 = f1_score(y, preds, zero_division=0)
    fα = wf1(y, proba)
    acc = accuracy_score(y, preds)
    print(f"{name:<11} P={pr:.3f} R={rc:.3f} F1={f1:.3f} Fα2={fα:.3f} Acc={acc:.3f}")
    return dict(p=pr, r=rc, f1=f1, fα=fα, acc=acc, proba=proba, preds=preds, y=y)

print("\n📊 Metrics (0.5 threshold)")
train_res = evaluate(X_tr_w, y_tr_w, "Train")
val_res = evaluate(X_va_w, y_va_w, "Val")
test_res = evaluate(X_te_w, y_te_w, "Test")

# ─── visualizations ─────────────────────────────────────────────────
print("\n📈 Creating visualizations...")

# 1. Training history
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes[0,0].plot(history.history['loss'], label='Train')
axes[0,0].plot(history.history['val_loss'], label='Val')
axes[0,0].set_title('Loss'); axes[0,0].legend()

axes[0,1].plot(history.history['accuracy'], label='Train')
axes[0,1].plot(history.history['val_accuracy'], label='Val')
axes[0,1].set_title('Accuracy'); axes[0,1].legend()

axes[1,0].plot(history.history['precision'], label='Train')
axes[1,0].plot(history.history['val_precision'], label='Val')
axes[1,0].set_title('Precision'); axes[1,0].legend()

axes[1,1].plot(history.history['recall'], label='Train')
axes[1,1].plot(history.history['val_recall'], label='Val')
axes[1,1].set_title('Recall'); axes[1,1].legend()

plt.tight_layout()
plt.savefig(OUT_DIR / f"training_history_{STAMP}.png", dpi=150)
plt.close()

# 2. Confusion Matrix (Test Set)
cm = confusion_matrix(test_res['y'], test_res['preds'])
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix - Test Set')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.savefig(OUT_DIR / f"confusion_matrix_{STAMP}.png", dpi=150)
plt.close()

# ─── save JSON summary ─────────────────────────────────────────────────
json.dump(
    {"params": BEST_PARAMS,
     "train": {k: v for k, v in train_res.items() if not isinstance(v, np.ndarray)},
     "val": {k: v for k, v in val_res.items() if not isinstance(v, np.ndarray)},
     "test": {k: v for k, v in test_res.items() if not isinstance(v, np.ndarray)}},
    open(OUT_DIR / f"results_{STAMP}.json", "w"), indent=2
)

print(f"\n✅ All artifacts saved in {OUT_DIR} ({STAMP})")
print(f"   - Model: final_model_{STAMP}.keras")
print(f"   - Scaler: scaler_{STAMP}.pkl")
print(f"   - Results: results_{STAMP}.json")
print(f"   - Plots: *_{STAMP}.png")


In [ ]:
{
  "arch": "conv_lstm",
  "window": 66,
  "filters": 32,
  "kernel": 5,
  "units": 32,
  "conv_drop": 0.1990,
  "lstm_drop": 0.0056,
  "dense": 32,
  "dropout": 0.3674,
  "l2": 1.73e-06,
  "lr": 5.455e-05,
  "batch": 32,
  "act": "relu",
  "pool": "gap",
  "conv_blocks": 2,
  "optim": "nadam",
  "precision": 0.554,
  "recall": 0.501,
  "f_alpha": 0.518
}

In [ ]:
{
  "arch": "conv_lstm",
  "window": 72,
  "filters": 96,
  "kernel": 4,
  "units": 32,
  "conv_drop": 0.2349,
  "lstm_drop": 0.0741,
  "dense": 128,
  "dropout": 0.3562,
  "l2": 1.93e-06,
  "lr": 8.557e-05,
  "batch": 128,
  "act": "relu",
  "pool": "gmp",
  "conv_blocks": 2,
  "optim": "nadam",
  "precision": 0.547,
  "recall": 0.371,
  "f_alpha": 0.415
}

In [ ]:
{
  "arch": "lstm_conv",
  "window": 66,
  "filters": 64,
  "kernel": 3,
  "units": 64,
  "conv_drop": 0.2411,
  "lstm_drop": 0.0560,
  "dense": 32,
  "dropout": 0.3584,
  "l2": 8.995e-06,
  "lr": 7.846e-05,
  "batch": 128,
  "act": "elu",
  "pool": "gmp",
  "conv_blocks": 1,
  "optim": "adamw",
  "precision": 0.533,
  "recall": 0.722,
  "f_alpha": 0.646
}